In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *


VISUALIZE = True
graph = None
SAMPLED_DATA = defaultdict(dict)
N_retry_test = 100

gtimer = GlobalTimer.instance()
crob = CombinedRobot(connection_list=(False, False))

GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"
try: os.mkdir(DATA_PATH)
except: pass

connection_list
(False, False)


## Load Global params

In [2]:
DATASET_LIST = sorted(os.listdir(DATA_PATH))

In [3]:
CURRENT_PATH = os.path.join(DATA_PATH, DATASET_LIST[0])
GLOBAL_PARAMS = load_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME))
for k, v in GLOBAL_PARAMS.items():
    locals()[k] = v

In [4]:
CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH 
Nw, Nd, Nh = Nwdh = int(Ws/L_CELL), int(Ds/L_CELL), int(Hs/L_CELL)
L_MAX = L_CELL*RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw*Nd*Nh, Nw, Nd, Nh))

scene size: 3375 (15,15,15)


## Load world

In [5]:
WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))

In [6]:
WORLD_PATH = os.path.join(CURRENT_PATH, WORLD_LIST[0])
SAMPLED_DATA["WORLD"] = load_json(os.path.join(WORLD_PATH, WORLD_FILENAME))
Trbt_dict = SAMPLED_DATA["WORLD"]["Trbt_dict"]
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

In [7]:
cam = None
# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict, crob.custom_limits, start_rviz=VISUALIZE)
ghnd = GeometryHandle(urdf_content)
time.sleep(2)
# set graph
if graph is None:
    graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                            urdf_content=urdf_content, combined_robot=crob)
    graph.set_camera(cam)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    if VISUALIZE: graph.set_rviz()

    # start UI
    ui_broker = UIBroker(graph)
    ui_broker.start_server()

    # set rviz
    if VISUALIZE: graph.set_rviz(crob.home_pose)
    ui_broker.set_tables()
else:
    graph.clear_markers()
    graph.clear_highlight()
    graph.ghnd.clear()
    graph.__init__(ghnd=graph.ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                   urdf_content=urdf_content, combined_robot=crob)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    if VISUALIZE: graph.set_rviz()
        
for gripper in GRIPPER_REFS.values():
    graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0,0,0], link_name=gripper['link_name'])
graph.register_binder(name='base', _type=PlaceFrame, point=[0,0,0], rpy=[0,0,0], link_name=BASE_LINK)
vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                                             dims=(0,0,0), center=(0,0,0), rpy=(0,0,0), collision=False, display=False
                                            )
graph.add_object("virtual",
                 SingleHandleObject(_object=vtem,
                                   action_point=FramePoint(name="point", _object=vtem, point=(0,0,0), rpy=(0,0,0), name_full=None)),
                 binding=("point", "base"))

obj_list=[]
col_obj_list=[]

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Load scene

In [8]:
SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))

In [9]:
SCENE_PATH = os.path.join(WORLD_PATH, SCENE_LIST[0])
SAMPLED_DATA["SCENE"] = load_json(os.path.join(SCENE_PATH, SCENE_FILENAME))
Q_s = np.array(SAMPLED_DATA["SCENE"]["Q_s"])
Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph, Q_s_loaded=Q_s)
Q_s_dict = SAMPLED_DATA["SCENE"]["Q_s_dict"]
obj_dat = SAMPLED_DATA["SCENE"]["obj_dat"]

In [10]:
if VISUALIZE:
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
for obj in obj_list: graph.remove_geometry(obj)
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name,*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

if "col_obj_list" in locals():
    for obj in col_obj_list: graph.remove_geometry(obj)

if VISUALIZE: graph.set_rviz()
dcol = DataCollector(graph, GRIPPER_REFS, S_F_RATIO=S_F_RATIO)
if VISUALIZE: graph.set_rviz()

# planners
mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names, binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path, graph.link_names, graph.combined_robot.robot_names)

## Load action

In [11]:
ACTION_LIST = sorted(filter(lambda x: x!=SCENE_FILENAME, os.listdir(SCENE_PATH)))

In [12]:
dcol.snode_dict = {int(k):v for k,v in load_json(os.path.join(SCENE_PATH, ACTION_LIST[2])).items()}

In [13]:
print([dcol.snode_dict[skey]['success'] for skey in sorted(dcol.snode_dict.keys())])

[True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False]


In [14]:
snode = dcol.snode_dict[0]
rname, inhand, obj, tar, dims_bak, color_bak, succ, _ = load_manipulation_from_dict(snode, graph.ghnd)

In [15]:
success_now = False
for _ in range(N_retry_test):
    if rname and tar: # handover case
        trajectory, Q_last, error, success_now = test_handover(graph, GRIPPER_REFS, rname, inhand, obj, tar, Q_s, dual_mplan_dict[(rname, tar)])
        remove_map = [[], [0,1]]
    elif inhand.collision: # place case
        trajectory, Q_last, error, success_now = test_place(graph, GRIPPER_REFS, rname, inhand, obj, tar, Q_s, mplan)
        remove_map = [[], [0,1]]
    elif obj.collision: # pick case
        trajectory, Q_last, error, success_now = test_pick(graph, GRIPPER_REFS, rname, inhand, obj, tar, Q_s, mplan)
        remove_map = [[1], [0]]
    else:
        raise(RuntimeError("non-implemented case"))
    if success_now:
        break
    else:
        print("fail log")
        
print("DATA CHECK {}: ({}->{})".format("SUCCESS" if succ == success_now else "FAILURE", succ, success_now))

if VISUALIZE and success_now:
    show_manip_coords(graph, GRIPPER_REFS, "TESTING", rname, inhand, obj, rname2=tar)
    graph.show_motion(trajectory, period=0.1)
    
remove1 = [[inhand, obj][iii] for iii in remove_map[0]]
remove2 = [[inhand, obj][iii] for iii in remove_map[1]]
reset_rendering(graph, "TESTING", remove1, remove2, dims_bak, color_bak, sleep=True, vis=True)

fail log
DATA CHECK SUCCESS: (True->True)


In [16]:
graph.set_rviz()

generate table - Geometrygenerate table - Handle

generate table - Binder
generate table - Object
